# PT data

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

2024-04-08 16:31:39.579322: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 16:31:39.579355: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 16:31:39.580324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 16:31:39.586490: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
GLC24_PA_metadata_train = pd.read_pickle("processed_data/GLC24_PA_metadata_train_elevation.pkl")

In [3]:
with pd.option_context('use_inf_as_na', True):
    display(GLC24_PA_metadata_train.isna().sum())

/tmp/ipykernel_11094/421287792.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('use_inf_as_na', True):


lon                       0
lat                       0
year                      0
geoUncertaintyInM     12496
areaInM2             234864
region                    0
country                   0
speciesId                 0
surveyId                  0
elevation                 0
dtype: int64

In [4]:
GLC24_PA_metadata_train = GLC24_PA_metadata_train.drop(columns=['geoUncertaintyInM'])

In [5]:
with pd.option_context('use_inf_as_na', True):
    GLC24_PA_metadata_train.areaInM2 = GLC24_PA_metadata_train.areaInM2.fillna(GLC24_PA_metadata_train.areaInM2.mean())

/tmp/ipykernel_11094/3152395630.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('use_inf_as_na', True):


In [6]:
with pd.option_context('use_inf_as_na', True):
    display(GLC24_PA_metadata_train.isna().sum())

/tmp/ipykernel_11094/421287792.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('use_inf_as_na', True):


lon          0
lat          0
year         0
areaInM2     0
region       0
country      0
speciesId    0
surveyId     0
elevation    0
dtype: int64

In [7]:
GLC24_PA_metadata_train.head()

,lon,lat,year,areaInM2,region,country,speciesId,surveyId,elevation
0,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,6874.0,212,26.0
1,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,476.0,212,26.0
2,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,11157.0,212,26.0
3,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,8784.0,212,26.0
4,3.099038,43.134956,2021,100.0,MEDITERRANEAN,France,4530.0,212,26.0


In [8]:
GLC24_PA_metadata_train.speciesId = GLC24_PA_metadata_train.speciesId.astype('uint16')

In [9]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(dtype='uint8')

In [10]:
transformed = ohe.fit_transform(GLC24_PA_metadata_train[['region', 'country']])


In [11]:
np.concatenate((ohe.categories_[0], ohe.categories_[1]), axis=0)

array(['ALPINE', 'ATLANTIC', 'BLACK SEA', 'BOREAL', 'CONTINENTAL',
       'MEDITERRANEAN', 'PANNONIAN', 'STEPPIC', 'Andorra', 'Austria',
       'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia',
       'Czech Republic', 'Denmark', 'France', 'Germany', 'Greece',
       'Hungary', 'Ireland', 'Italy', 'Latvia', 'Luxembourg', 'Monaco',
       'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Spain',
       'Switzerland', 'The former Yugoslav Republic of Macedonia'],
      dtype=object)

In [12]:
GLC24_PA_metadata_train[np.concatenate((ohe.categories_[0], ohe.categories_[1]), axis=0)] = transformed.toarray()
print(GLC24_PA_metadata_train.head())

        lon        lat  year  areaInM2         region country  speciesId  \
0  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France       6874   
1  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France        476   
2  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France      11157   
3  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France       8784   
4  3.099038  43.134956  2021     100.0  MEDITERRANEAN  France       4530   

   surveyId  elevation  ALPINE  ...  Norway  Poland  Portugal  Romania  \
0       212       26.0       0  ...       0       0         0        0   
1       212       26.0       0  ...       0       0         0        0   
2       212       26.0       0  ...       0       0         0        0   
3       212       26.0       0  ...       0       0         0        0   
4       212       26.0       0  ...       0       0         0        0   

   Serbia  Slovakia  Slovenia  Spain  Switzerland  \
0       0         0         0      0         

In [13]:
GLC24_PA_metadata_train = GLC24_PA_metadata_train.drop(columns=['region', 'country'])

In [14]:
transformed = ohe.fit_transform(GLC24_PA_metadata_train[['speciesId']])

In [15]:
GLC24_PA_metadata_train[ohe.categories_[0]] = transformed.toarray()
print(GLC24_PA_metadata_train.head())

/tmp/ipykernel_11094/226669144.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  GLC24_PA_metadata_train[ohe.categories_[0]] = transformed.toarray()
/tmp/ipykernel_11094/226669144.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  GLC24_PA_metadata_train[ohe.categories_[0]] = transformed.toarray()
/tmp/ipykernel_11094/226669144.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

        lon        lat  year  areaInM2  speciesId  surveyId  elevation  \
0  3.099038  43.134956  2021     100.0       6874       212       26.0   
1  3.099038  43.134956  2021     100.0        476       212       26.0   
2  3.099038  43.134956  2021     100.0      11157       212       26.0   
3  3.099038  43.134956  2021     100.0       8784       212       26.0   
4  3.099038  43.134956  2021     100.0       4530       212       26.0   

   ALPINE  ATLANTIC  BLACK SEA  ...  11239  11240  11241  11242  11243  11245  \
0       0         0          0  ...      0      0      0      0      0      0   
1       0         0          0  ...      0      0      0      0      0      0   
2       0         0          0  ...      0      0      0      0      0      0   
3       0         0          0  ...      0      0      0      0      0      0   
4       0         0          0  ...      0      0      0      0      0      0   

   11246  11248  11253  11254  
0      0      0      0      0  
1   

In [16]:
# save GLC24_PA_metadata_train pickled
GLC24_PA_metadata_train.to_pickle("processed_data/GLC24_PA_metadata_train_elevation_onehot.pkl")

In [ ]:
GLC24_PA_metadata_train_grouped = GLC24_PA_metadata_train.groupby('surveyId').max().copy()

In [ ]:
GLC24_PA_metadata_train_grouped.to_pickle("processed_data/GLC24_PA_metadata_train_elevation_onehot_grouped.pkl")